# Modeling Crop Yield Anomaly
## Python modules

In [1]:
## import warnings
import time
import math
import os
import glob
from pyhere import here
from datetime import date
import re

import numpy as np
import pandas as pd
import geopandas

import pyarrow
import itertools
import multiprocessing
import p_tqdm

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split, KFold, LeaveOneGroupOut, cross_val_score, GridSearchCV, cross_val_predict
from sklearn.metrics import r2_score
from scipy.stats import spearmanr,  pearsonr

In [2]:
def str2bool(string):
    return string.lower() in ("yes", "true", "t", "1")

point_pattern = re.compile("20k-points")
wa_pattern = re.compile("cm-False")

In [3]:
data_dir = here("data")
directory = here("data", "random_features", "summary")
today = date.today().strftime("%Y-%m-%d")
files = os.listdir(directory)
files = [f for f in files if f not in ('.gitkeep', '.ipynb_checkpoints')]
files = [f for f in files if not (bool(point_pattern.search(f)) & bool(wa_pattern.search(f)))]
len(files)

44

In [4]:
crop_df = pd.read_csv(here('data', 'crop_yield', 'cfs_maize_districts_zambia_2009_2022.csv'))
crop_df = crop_df.set_index(['district', 'year'])[['log_yield_mt_anomaly']]

In [5]:
def model(params):
#########################################     SET PARAMS    #########################################
    file         = params
    f            = file.split(sep="_")
    satellite    = f[0]
    bands        = f[1].replace("bands-", "")
    country_code = f[2]
    points       = f[3].replace("k-points", "")
    num_features = f[4].replace("-features", "")
    yrs          = f[5].replace("yr-", "").split(sep="-")
    mns          = f[6].replace("mn-", "").split(sep="-")
    limit_months = str2bool(f[7].replace("lm-", ""))
    crop_mask    = str2bool(f[8].replace("cm-", ""))
    weighted_avg = str2bool(f[9].replace("wa-", ""))
    years        = range(int(yrs[0]), int(yrs[1])+1)
    month_range  = list(range(int(mns[0]), int(mns[1])+1))
    
    alphas = {'alpha': np.logspace(-8, 8, base = 10, num = 17)}
    kfold = KFold()
    logo = LeaveOneGroupOut()
    ridge = Ridge() 
#########################################     READ DATA    #########################################
    fn = f"{directory}/{file}"
    features = pd.read_feather(fn)
     
    drop_cols = ['district', 'year', 'yield_mt', 'crop_perc']
            
    if weighted_avg:
        drop_cols.remove("crop_perc")
    else:
        pass

#########################################     CALCULATE ANOMALY   #########################################
    features['yield_mt'] = np.log10(features['yield_mt'] + 1)
    features.set_index(['year', 'district'], inplace=True)
    var_cols = features.columns
    features = features[var_cols] - features.groupby(['district'], as_index=True)[var_cols].transform('mean')
    features.reset_index(drop=False, inplace=True)

#########################################     K-FOLD SPLIT    #########################################
    x_all = features.drop(drop_cols, axis = 1) 
    y_all = features.yield_mt
    x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=0)
    
#########################################     K-FOLD  CV   ###########################################
    ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
    kfold_ridge_reg = GridSearchCV(ridge, alphas, scoring = 'r2', cv = kfold)
    kfold_ridge_reg.fit(x_train, y_train)
    kfold_best_model = kfold_ridge_reg.best_estimator_
    ### PREDICT - PREDICTING WITH BEST HYPERPARAMETER
    kfold_val_predictions = cross_val_predict(kfold_best_model, X = x_train, y = y_train, cv = kfold)   
    y_pred_train_k = kfold_best_model.predict(x_train)
    y_pred_test_k  = kfold_best_model.predict(x_test)

#########################################     LOGO ITERATOR   ###########################################
    logo_val_results   = []
    logo_train_results = []
    logo_test_results  = []

    for year in features.year.unique():
#########################################     LOGO SPLIT   ###########################################
        logo_x_train = features[features.year != year].drop(drop_cols, axis=1)
        logo_y_train = features[features.year != year].yield_mt.ravel()
        logo_g_train = features[features.year != year].year.ravel()
        logo_d_train = features[features.year != year].district.ravel()

        logo_x_test = features[features.year == year].drop(drop_cols, axis=1)
        logo_y_test = features[features.year == year].yield_mt.ravel()
        logo_g_test = features[features.year == year].year.ravel()
        logo_d_test = features[features.year == year].district.ravel()

#########################################     LOGO CV   ###########################################
        ### GRID SEARCH - FINDING BEST REGULARIZATION PARAMETER
        logo_ridge_reg = GridSearchCV(ridge, alphas, scoring='r2', cv=logo)
        logo_ridge_reg.fit(logo_x_train, logo_y_train, groups=logo_g_train)
        logo_best_model = logo_ridge_reg.best_estimator_
        ### PREDICT - PREDICTING WITH BEST HYPERPARAMETER
        logo_val_predictions = cross_val_predict(logo_best_model, X=logo_x_train, y=logo_y_train, groups=logo_g_train, cv=logo) 
        logo_train_pred = logo_best_model.predict(logo_x_train)
        logo_test_pred  = logo_best_model.predict(logo_x_test)

#########################################     LOGO RESULTS   ###########################################
        val_results   = {'year': logo_g_train, 'district': logo_d_train, 'split': 'val', 
                         'observed': logo_y_train, 'predicted': logo_val_predictions}

        train_results = {'year': logo_g_train, 'district': logo_d_train,'split': 'train', 
                         'observed': logo_y_train, 'predicted': logo_train_pred}

        test_results  = {'year': logo_g_test, 'district': logo_d_test, 'split': 'test', 
                         'observed': logo_y_test, 'predicted': logo_test_pred}
        
        logo_val_results.append(val_results)
        logo_train_results.append(train_results)
        logo_test_results.append(test_results)

#########################################     EXPLODE RESULTS   ###########################################
    explode_cols = ['year', 'district', 'observed', 'predicted']
    val_df   = pd.DataFrame(logo_val_results  ).explode(explode_cols) 
    train_df = pd.DataFrame(logo_train_results).explode(explode_cols) 
    test_df  = pd.DataFrame(logo_test_results ).explode(explode_cols)
    
    group_cols = ['year', 'district', 'split']
    val_summary   =   val_df.groupby(group_cols, as_index=False).mean()
    train_summary = train_df.groupby(group_cols, as_index=False).mean()
    
#########################################     SAVE RESULTS    #########################################
    d = {
        'country': country_code,
        'satellite': satellite,
        'bands': bands,
        'num_features': num_features,
        'points': points, 
        'month_range': f'{min(month_range)}-{max(month_range)}',
        
        'limit_months': limit_months,
        'crop_mask': crop_mask,
        'weighted_avg': weighted_avg,
        
        'kfold_total_n': len(x_all),
        'kfold_train_n': len(x_train),
        'kfold_test_n' : len(x_test),
        
        'kfold_best_reg_param': list(kfold_ridge_reg.best_params_.values())[0],
        'kfold_mean_of_val_R2s': kfold_ridge_reg.best_score_,
        'kfold_val_R2': r2_score(y_train, kfold_val_predictions),
        'kfold_val_r' : pearsonr(kfold_val_predictions, y_train)[0],
        'kfold_val_r2': pearsonr(kfold_val_predictions, y_train)[0] ** 2,
        
        'kfold_train_R2': r2_score(y_train, y_pred_train_k),
        'kfold_train_r' : pearsonr(y_pred_train_k, y_train)[0],
        'kfold_train_r2': pearsonr(y_pred_train_k, y_train)[0] ** 2,
        
        'kfold_test_R2': r2_score(y_test, y_pred_test_k),
        'kfold_test_r' : pearsonr(y_pred_test_k, y_test)[0],
        'kfold_test_r2': pearsonr(y_pred_test_k, y_test)[0] ** 2,
        
        'logo_total_n': len(features),
        'logo_train_n': len(train_df),
        'logo_test_n' : len(test_df),    
        
        'logo_best_reg_param': list(logo_ridge_reg.best_params_.values())[0],      
        'logo_summary_val_R2': r2_score(val_summary.observed, val_summary.predicted),
        'logo_summary_val_r' : pearsonr(val_summary.observed, val_summary.predicted)[0],
        'logo_val_R2' : r2_score(val_df.observed, val_df.predicted),
        'logo_val_r'  : pearsonr(val_df.predicted, val_df.observed)[0],
        'logo_val_r2' : pearsonr(val_df.predicted, val_df.observed)[0] ** 2,
        
        'logo_summary_train_R2': r2_score(train_summary.observed, train_summary.predicted),
        'logo_summary_train_r' : pearsonr(train_summary.observed, train_summary.predicted)[0],
        'logo_train_R2': r2_score(train_df.observed, train_df.predicted),
        'logo_train_r' : pearsonr(train_df.predicted, train_df.observed)[0],
        'logo_train_r2': pearsonr(train_df.predicted, train_df.observed)[0] ** 2,
        
        'logo_test_R2': r2_score(test_df.observed, test_df.predicted),
        'logo_test_r' : pearsonr(test_df.predicted, test_df.observed)[0],
        'logo_test_r2': pearsonr(test_df.predicted, test_df.observed)[0] ** 2,
    }
    return pd.DataFrame(data=d, index=[0])

In [ ]:
%%time     
##### With progress bar
workers = 44 ## os.cpu_count()
if __name__ == "__main__":
    output = []
    for result in p_tqdm.p_map(model, files, num_cpus=workers):
        output.append(result)
    results = pd.concat(output).reset_index(drop=True)
    today = date.today().strftime("%Y-%m-%d")
    file_name = f'anomaly_results_{today}.csv'
    print(f"Saving results as: {file_name}\n\n")           
    results.to_csv(here("data","results", file_name), INDEX = False)

  0%|          | 0/44 [00:00<?, ?it/s]

In [ ]:
# results